<a href="https://colab.research.google.com/github/mhask94/cs474_labs_f2019/blob/master/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 4: Cancer Detection

## Objective
* To build a dense prediction model
* To begin reading current papers in DNN research

## Deliverable
For this lab, you will turn in a notebook that describes your efforts at creating
a pytorch radiologist. Your final deliverable is a notebook that has (1) deep network,
(2) cost function, (3) method of calculating accuracy,
(4) an image that shows the dense prediction produced by your network on the pos_test_000072.png image.
This is an image in the test set that your network will not have seen before.
This image, and the ground truth labeling, is shown below.
(And is contained in the downloadable dataset below).

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=200&tok=a8ac31&media=cs501r_f2016:pos_test_000072_output.png)
<img src="http://liftothers.org/dokuwiki/lib/exe/fetch.php?media=cs501r_f2016:pos_test_000072.png" width="200">


## Grading standards
Your notebook will be graded on the following:
* 40% Proper design, creation and debugging of a dense prediction network
* 40% Proper implementation of a loss function and train/test set accuracy measure
* 10% Tidy visualizations of loss of your dense predictor during training
* 10% Test image output


## Data set
The data is given as a set of 1024×1024 PNG images. Each input image (in 
the ```inputs``` directory) is an RGB image of a section of tissue,
and there a file with the same name (in the ```outputs``` directory) 
that has a dense labeling of whether or not a section of tissue is cancerous
(white pixels mean “cancerous”, while black pixels mean “not cancerous”).

The data has been pre-split for you into test and training splits.
Filenames also reflect whether or not the image has any cancer at all 
(files starting with ```pos_``` have some cancerous pixels, while files 
starting with ```neg_``` have no cancer anywhere).
All of the data is hand-labeled, so the dataset is not very large.
That means that overfitting is a real possibility.

## Description
For a video including some tips and tricks that can help with this lab: [https://youtu.be/Ms19kgK_D8w](https://youtu.be/Ms19kgK_D8w)
For this lab, you will implement a virtual radiologist.
You are given images of possibly cancerous tissue samples, 
and you must build a detector that identifies where in the tissue cancer may reside.

---

### Part 0
Watch and follow video tutorial:

https://youtu.be/Ms19kgK_D8w

**TODO:**

**DONE:**
* Watch tutorial

### Part 1
Implement a dense predictor

In previous labs and lectures, we have talked about DNNs that classify an 
entire image as a single class. Here, however, we are interested in a more nuanced classification: 
given an input image, we would like to identify each pixel that is possibly cancerous. 
That means that instead of a single output, your network should output an “image”, 
where each output pixel of your network represents the probability that a pixel is cancerous.

**TODO:**

* Create a Network that classifies each pixel as a 1 or 0 for cancerous / not cancerous

**DONE:**

___

### Part 1a
Implement your network topology


Use the “Deep Convolution U-Net” from this paper: [(U-Net: Convolutional Networks for Biomedical Image Segmentation)](https://arxiv.org/pdf/1505.04597.pdf) 

![(Figure 1)](https://lh3.googleusercontent.com/qnHiB3B2KRxC3NjiSDtY08_DgDGTDsHcO6PP53oNRuct-p2QXCR-gyLkDveO850F2tTAhIOPC5Ha06NP9xq1JPsVAHlQ5UXA5V-9zkUrJHGhP_MNHFoRGnjBz1vn1p8P2rMWhlAb6HQ=w2400)

You should use existing pytorch functions (not your own Conv2D module), such as ```nn.Conv2d```;
you will also need the pytorch function ```torch.cat``` and ```nn.ConvTranspose2d```

```torch.cat``` allows you to concatenate tensors.
```nn.ConvTranspose2d``` is the opposite of ```nn.Conv2d```.
It is used to bring an image from low res to higher res.
[This blog](https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0) should help you understand this function in detail.

Note that the simplest network you could implement (with all the desired properties)
is just a single convolution layer with two filters and no relu! 
Why is that? (of course it wouldn't work very well!)

**TODO:**

**DONE:**
* Understand the U-Net architecture
* Understand concatenation of inputs from multiple prior layers
* Understand ConvTranspose
* Answer Question / Reflect on simplest network with the desired properties
- Because 2 filters would provide the 2 channels needed in the end, so it would provide the right output size.
___
The intention of this lab is to learn how to make deep neural nets and implement loss function.
Therefore we'll help you with the implementation of Dataset.
This code will download the dataset for you so that you are ready to use it and focus on network
implementation, losses and accuracies.

In [1]:
!pip3 install torch --user
!pip3 install torchvision --user
!pip3 install tqdm --user

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

In [0]:
class CancerDataset(Dataset):
  def __init__(self, root, download=True, size=512, train=True):
    if download and not os.path.exists(os.path.join(root, 'cancer_data')):
      datasets.utils.download_url('http://liftothers.org/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      self.extract_gzip(os.path.join(root, 'cancer_data.tar.gz'))
      self.extract_tar(os.path.join(root, 'cancer_data.tar'))
    
    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'cancer_data', 'cancer_data')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'inputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'outputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  @staticmethod
  def extract_gzip(gzip_path, remove_finished=False):
    print('Extracting {}'.format(gzip_path))
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())
    if remove_finished:
      os.unlink(gzip_path)
  
  @staticmethod
  def extract_tar(tar_path):
    print('Untarring {}'.format(tar_path))
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]
  
  def __len__(self):
    return len(self.dataset_folder)

___

### Part 1b
Implement a cost function

You should still use cross-entropy as your cost function, but you may need to think hard about how exactly to set this up – your network should output cancer/not-cancer probabilities for each pixel, which can be viewed as a two-class classification problem.

**TODO:**

**DONE:**
* Adapt CrossEntropyLoss for 2 class pixel classification



In [3]:
# You'll probably want a function or something to test input / output sizes of the ConvTranspose2d layer
def testConvTranspose2dSize():
  x_test = torch.zeros(1,3,512,512)
  down = nn.Conv2d(3, 64, (3,3), padding=1, stride=2)
  out_test = down(x_test)
  print('down: \n',out_test.size())

  up = nn.ConvTranspose2d(3, 64, (2,2), padding=0, stride=2)
  up_test = up(x_test)
  print('up: \n',up_test.size())
  
testConvTranspose2dSize()

down: 
 torch.Size([1, 64, 256, 256])
up: 
 torch.Size([1, 64, 1024, 1024])


In [0]:
# You are welcome (and encouraged) to use the built-in batch normalization and dropout layer.

class UnetBlock(nn.Module):
  def __init__(self,c_in, c_out, kernel=(3,3), pad=(1,1), after='none'):
    super(UnetBlock, self).__init__()    
    self.net = nn.Sequential(
        nn.Conv2d(c_in, c_out, kernel, padding=pad),
        nn.ReLU(),
        nn.Conv2d(c_out, c_out, kernel, padding=pad),
        nn.ReLU(),
    )
    self.after = after
    if after == 'up':
      self.net2 = nn.ConvTranspose2d(c_out, c_out//2, (2,2), padding=0, stride=2)
    elif after == 'down':
      self.net2 = nn.Conv2d(c_out, c_out, kernel, padding=pad, stride=2)
    elif after == 'end':
      self.net2 = nn.Conv2d(c_out, 2, (1,1), padding=0)
    else: # none
      self.net2 = None
    
  def forward(self, x):
    # (n, c, h, w) sqeeze 2nd dim => (n, c, w) squeeze 2nd dim => (n, c)
    net_out = self.net(x).squeeze(2).squeeze(2)
    after_out = self.net2(net_out).squeeze(2).squeeze(2)
    if self.after == 'down':
      return net_out, after_out
    else:
      return after_out

class UnetCancerDetection(nn.Module):
  def __init__(self):
    super(UnetCancerDetection, self).__init__()
    self.down1 = UnetBlock(3, 64, after='down')
    self.down2 = UnetBlock(64, 128, after='down')
    self.down3 = UnetBlock(128, 256, after='down')
    self.down4 = UnetBlock(256, 512, after='down')
    self.up1 = UnetBlock(512, 1024, after='up')
    self.up2 = UnetBlock(1024, 512, after='up')
    self.up3 = UnetBlock(512, 256, after='up')
    self.up4 = UnetBlock(256, 128, after='up')
    self.end = UnetBlock(128, 64, after='end')
 
  def forward(self, x):
    skip1, down1 = self.down1(x)
    skip2, down2 = self.down2(down1)
    skip3, down3 = self.down3(down2)
    skip4, down4 = self.down4(down3)
    up1 = self.up1(down4)
    up2 = self.up2(torch.cat((skip4, up1), dim=1))
    up3 = self.up3(torch.cat((skip3, up2), dim=1))
    up4 = self.up4(torch.cat((skip2, up3), dim=1))
    out = self.end(torch.cat((skip1, up4), dim=1))
    
#     exp = torch.exp(end)
#     shape = list(exp.shape)
#     shape[1] = 1
#     out = exp / torch.sum(exp, dim=1).reshape((*shape))

    return out


In [5]:
# Create your datasets and neural network as you have before
gc.collect()

try:
  train_dataset = CancerDataset('/tmp/cancerdata', train=True)
  val_dataset = CancerDataset('/tmp/cancerdata', train=False)

  train_loader = DataLoader(train_dataset, batch_size=7, pin_memory=True,
                            num_workers=2, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=7, pin_memory=True,
                          num_workers=2)

  model = UnetCancerDetection()
  model.cuda()

  objective = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=1e-4)

  print('CUDA mem: ', torch.cuda.memory_allocated(0) / 1e9)
  
except:
  __ITB__()

0it [00:00, ?it/s]

100%|█████████▉| 2750210048/2750494655 [02:24<00:00, 21562464.97it/s]

Extracting /tmp/cancerdata/cancer_data.tar.gz


2750496768it [02:40, 21562464.97it/s]                                

Untarring /tmp/cancerdata/cancer_data.tar
CUDA mem:  0.138633216


In [6]:
def testNetworkOutputSizes():
  x_test = torch.zeros(1,3,512,512)

  block1 = UnetBlock(3, 32, after='down')
  skip, after = block1(x_test)
  print('skip: ', skip.size())
  print('after: ', after.size())

  block2 = UnetBlock(3, 32, after='up')
  up_after = block2(x_test)
  print('up_after: ', up_after.size())

  net = UnetCancerDetection()
  out = net(x_test)
  print('out: ', out.size())
  print(out[0,:,0,0])
  print(out[0,:,5,5])
  
testNetworkOutputSizes()

skip:  torch.Size([1, 32, 512, 512])
after:  torch.Size([1, 32, 256, 256])
up_after:  torch.Size([1, 16, 1024, 1024])
out:  torch.Size([1, 2, 512, 512])
tensor([0.0909, 0.0324], grad_fn=<SelectBackward>)
tensor([0.0871, 0.0314], grad_fn=<SelectBackward>)


In [0]:
# This is what was talked about in the video for memory management
def calcAccuracy(yhat, ytruth):
  exp = torch.exp(yhat)
  shape = list(exp.shape)
  shape[1] = 1
  cancer_mask = (exp / torch.sum(exp, dim=1).reshape((*shape)))[:,0] > 0.5
#   pdb.set_trace()
  num_same_pixels = torch.sum(cancer_mask.byte() ^ ytruth.byte()).item()
  accuracy = 1 - num_same_pixels / ytruth.size().numel()
  return accuracy

def trainBatch(x, ytruth):
  global model, optimizer, objective
  gc.collect()

  x, ytruth = x.cuda(async=True), ytruth.cuda(async=True)
  optimizer.zero_grad()
  yhat = model(x)

  loss = objective(yhat, ytruth.long())
  loss.backward()
  losses.append(loss.item())

  accuracy = calcAccuracy(yhat, ytruth)

  optimizer.step()

  return loss, accuracy

def validate():
  global val_loader, model, objective
  gc.collect()
  with torch.no_grad():
    total_val_loss = []
    total_val_acc = []
    for x,y in val_loader:
      val_yhat = model(x.cuda())
      val_y = y.cuda()
      
      val_loss = objective(val_yhat, val_y.long()).item()
      total_val_loss.append(val_loss)

      val_acc = calcAccuracy(val_yhat, val_y)
      total_val_acc.append(val_acc)

    val_loss = np.mean(total_val_loss)
    val_acc = np.mean(total_val_acc) 

    return val_loss, val_acc

losses = []
val_losses = []
accuracies = []
val_accuracies = []

try:
  for epoch in range(10):
    loop = tqdm(total=len(train_loader),position=0,leave=False)
    for batch, (x,y_truth) in enumerate(train_loader):
      gc.collect()
      loss, accuracy = trainBatch(x, y_truth)
      losses.append(loss)
      accuracies.append(accuracy)
      
      mem = torch.cuda.memory_allocated(0) / 1e9
      string = 'e:{}, l:{:.4f}, a:{:.3f}, m:{:.2f} '
      string = string.format(epoch,loss,accuracy,mem)
      loop.set_description(string)
      loop.update(1)

      if batch % 100 == 0:
        val_loss, val_acc = validate()
        val_losses.append((len(losses), val_loss))
        val_accuracies.append((len(accuracies), val_acc))
    loop.close()
  batch_nums, vals = zip(*val_losses)
  vbatch_nums, vaccs = zip(*val_accuracies)  
except:
  __ITB__()

e:1, l:0.2520, a:0.062, m:0.55 :  20%|██        | 39/192 [04:11<13:08,  5.15s/it]


___

### Part 2

Plot performance over time

Please generate a plot that shows loss on the training set as a function of training time. Make sure your axes are labeled!

**TODO:**

**DONE:**
* Plot training loss as function of training time (not Epochs)


In [0]:
# Your plotting code here
plt.figure(1)
plt.plot(losses, label='train')
plt.plot(batch_nums, vals, label='val')
plt.ylabel('Loss')
plt.xlabel('Batch')
plt.legend()

plt.figure(2)
plt.plot(accuracies, label='train')
plt.plot(vbatch_nums, vaccs, label='val')
plt.ylabel('Accuracy')
plt.xlabel('Batch')
plt.legend()

plt.show()

___

### Part 3

Generate a prediction on the pos_test_000072.png image

Calculate the output of your trained network on the pos_test_000072.png image,
then make a hard decision (cancerous/not-cancerous) for each pixel.
The resulting image should be black-and-white, where white pixels represent things
you think are probably cancerous.

**TODO:**

**DONE:**

**NOTE:**

Guessing that the pixel is not cancerous every single time will give you an accuracy of ~ 85%.
Your trained network should be able to do better than that (but you will not be graded on accuracy).
This is the result I got after 1 hour or training.

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=d23e0b&media=cs501r_f2016:training_accuracy.png)
![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=bb8e3c&media=cs501r_f2016:training_loss.png)

In [0]:
def testModel():
  x,truth_img = val_dataset[172]
  x = x.cuda(async=True)
  out = model(x.unsqueeze(0))
  
  exp = torch.exp(out)
  shape = list(exp.shape)
  shape[1] = 1
  img = (exp / torch.sum(exp, dim=1).reshape((*shape)))[:,0] > 0.5
  print('shape: ', img.shape)
  fig, ax = plt.subplots()
  im1 = ax.imshow(img[0].cpu())
  im2 = ax.imshow(truth_img)
#   ax.set_title('Test Image')
  plt.show()  
  
testModel()

In [0]:
# Code for testing prediction on an image
img,test = val_dataset[172]

out_img = model(img.unsqueeze(0))

fig, ax = plt.subplots()
im = ax.imshow(test)
ax.set_title('Test Image')
plt.show()